In [1]:
from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirect, ForecasterDirectMultiVariate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

In [2]:
rng = np.random.default_rng(123)
y = pd.Series(rng.normal(size=10000), index=pd.date_range("2020-01-01", periods=10000, freq="h"))
y_original = y.copy()
forecaster = ForecasterRecursive(
    regressor=LinearRegression(),
    lags=100
)

In [3]:
%%timeit
forecaster._create_lags(
    y
)

5.9 ms ± 171 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
x_train, y_train = forecaster.create_train_X_y(y=y)
x_train.iloc[1,1] = 10000
pd.testing.assert_series_equal(y, y_original)

In [5]:
%%timeit
forecaster.fit(y=y)

224 ms ± 39.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
rng = np.random.default_rng(123)
series = {
    f"series_{i}": pd.Series(
        rng.normal(size=1000), index=pd.date_range("2020-01-01", periods=1000)
    )
    for i in range(1000)
}
forecaster = ForecasterRecursiveMultiSeries(
    regressor=LinearRegression(),
    lags=3,
)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : skforecast.exceptions.DataTransformationWarning                           │
│ Location :                                                                           │
│ /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skfore │
│ cast/recursive/_forecaster_recursive_multiseries.py:478                              │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [7]:
%%timeit

for y in series.values():
    forecaster._create_lags(
        y
    )

147 ms ± 6.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit

forecaster.fit(series=series)

2.17 s ± 32.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
from skforecast.direct import ForecasterDirect

forecaster = ForecasterDirect(
    regressor=LinearRegression(),
    steps=24,
    lags=50
)
rng = np.random.default_rng(123)
y = pd.Series(rng.normal(size=1000), index=pd.date_range("2020-01-01", periods=1000))

In [10]:
%%timeit
forecaster._create_lags(
    y
)

2.59 ms ± 78.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%%timeit
forecaster.fit(y=y)

176 ms ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
forecaster = ForecasterDirectMultiVariate(
    regressor=LinearRegression(),
    steps=24,
    lags=50,
    level = "series_1"
)
rng = np.random.default_rng(123)
series = {
    f"series_{i}": pd.Series(
        rng.normal(size=1000), index=pd.date_range("2020-01-01", periods=1000)
    )
    for i in range(20)
}
series = pd.DataFrame(series)

In [18]:
%%timeit
forecaster._create_train_X_y(
    series = series,
)

12.8 ms ± 150 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
forecaster.fit(series=series)

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd

def is_view(obj1, obj2):
    """
    Check if obj2 is a view of obj1.
    Works for NumPy arrays, pandas Series, and pandas DataFrames.
    """
    # Convert pandas objects to underlying NumPy arrays
    if isinstance(obj1, (pd.Series, pd.DataFrame)):
        arr1 = obj1.values
    else:
        arr1 = obj1

    if isinstance(obj2, (pd.Series, pd.DataFrame)):
        arr2 = obj2.values
    else:
        arr2 = obj2

    # Use NumPy function to check shared memory
    return np.shares_memory(arr1, arr2)
